In [1]:
import numpy as np, pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  

import requests
import time, logging, os, requests

import datetime
from datetime import timedelta
import statsmodels.api as sm

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit


from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

import lightgbm as lgb

import warnings
import itertools

/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
!ls

1-1-lgb.ipynb	 lstm_predictor.py	    train_1.csv
11_stack.ipynb	 main.ipynb		    train.csv
1-2.ipynb	 plot			    train.csv.zip
ATM.csv		 __pycache__		    trainmon.csv
dates.csv	 RealMarketPriceDataPT.csv  Untitled1.ipynb
figure.pdf	 simple_stat_id.csv	    Untitled2.ipynb
Great sol.ipynb  simple_stat_id.csv.zip     Папка
hol.csv		 submission.csv
IDAO-master	 Tensorflow.ipynb


## Load

In [3]:
df = pd.read_csv('train.csv .zip', index_col = 'DATE')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv .zip'

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
def score_series(y_true, y_pred):
    """
        Expects y_true and y_pred to be 1d arrays of the same lenght
    """
    return (np.sum(np.abs(y_pred-y_true))/np.sum(np.abs(y_true)))*10000

In [ ]:
scorer = make_scorer(score_series)

### Plot

In [ ]:
res = []
for i in df['ATM_ID'].unique():
    res.append([i, df[df['ATM_ID']==i]['CLIENT_OUT'].value_counts()[0], 
                df[df['ATM_ID']==i]['CLIENT_OUT'].mean(), df[df['ATM_ID']==i]['CLIENT_OUT'].median(), 
                df[df['ATM_ID']==i]['CLIENT_OUT'].std()])
    
    df[df['ATM_ID']==i]['CLIENT_OUT'].plot()
    plt.savefig('plot/id'+str(i)+'.png')
    plt.clf()

In [ ]:
res = pd.DataFrame(res, columns=['ATM_ID', 'NULL_C', 'MEAN', 'MEDIAN', 'STD'])

In [ ]:
res.sort_values(by='NULL_C').to_csv('simple_stat_id.csv', index=False)

In [ ]:
res.sort_values(by='STD').head(50)

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
y = df[df['ATM_ID']==136]['CLIENT_OUT']
y.plot()

In [ ]:
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 1) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}1 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

(1, 1, 1)x(0, 0, 1, 1)1

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 0, 1),
                                seasonal_order=(1, 1, 1, 1),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
results.get_prediction()

In [ ]:
pred = results.get_prediction(start = pd.to_datetime('2017-08-15') ,end=pd.to_datetime('2017-09-17'), dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
ax = y['2017-06':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=0.5)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')
plt.legend()

plt.show()

In [ ]:
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_absolute_error

In [ ]:
def check_id(y):
    # split dataset
    X = y.values
    train, test = X[390:len(X)-360], X[len(X)-360:]
    
    # train autoregression
    model = AR(train)
    model_fit = model.fit()
    window = model_fit.k_ar
    coef = model_fit.params
    
    # walk forward over time steps in test
    history = train[len(train)-window:]
    history = [history[i] for i in range(len(history))]
    predictions = []
    for t in range(len(test)):
        length = len(history)
        lag = [history[i] for i in range(length-window,length)]
        yhat = coef[0]
        for d in range(window):
            yhat += coef[d+1] * lag[window-d-1]
        obs = test[t]
        predictions.append(yhat)
        history.append(obs)
    error = mean_absolute_error(test, predictions)
    return error

In [4]:
forc=[]
for i in df['ATM_ID'].unique():
    y = df[df['ATM_ID']==i]['CLIENT_OUT']
    forc.append([check_id(y), i])

NameError: name 'df' is not defined

In [ ]:
forc = pd.DataFrame(forc, columns=['ERROR', 'ATM_ID'])

In [ ]:
forc.sort_values(by='ERROR').head(50).to_csv('ATM.csv',index=False)

In [ ]:
def pred(y):
    model = AR(y)
    model_fit = model.fit()
    predictions = model_fit.predict(start = pd.to_datetime('2017-01-15') ,end=pd.to_datetime('2017-09-17'))
    return predictions[1:]

In [ ]:
forc.sort_values(by='ERROR').head(50)

In [ ]:
from fbprophet import Prophet